## A Simple CNN Baseline Using Pretrained ResNet18
Author: Junye Wang (群柴犬@DataTech工作室)

### 导入必要的库工具

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

### 适配竞赛服务器的文件路径

In [2]:
DATASET_PATH = '/kaggle/input/the-1st-data-tech-alchemist-cup'
TRAIN_CSV = os.path.join(DATASET_PATH, 'train.csv')
TEST_CSV = os.path.join(DATASET_PATH, 'test.csv')
TRAIN_IMAGES = os.path.join(DATASET_PATH, 'train_images')
TEST_IMAGES = os.path.join(DATASET_PATH, 'test_images')
SUBMISSION_FILE = '/kaggle/working/submission.csv'

### 定义超参数

In [3]:
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

### 加载数据集文件目录

In [4]:
def load_csv():
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    labels = train_df['label'].unique().tolist()
    label_to_idx = {label: idx for idx, label in enumerate(labels)}
    idx_to_label = {idx: label for label, idx in label_to_idx.items()}
    return train_df, test_df, label_to_idx, idx_to_label

### 自定义数据集类

In [5]:
class ButterflyDataset(Dataset):
    def __init__(self, dataframe, image_dir, label_to_idx=None, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.label_to_idx = label_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.label_to_idx:
            label = self.dataframe.iloc[idx, 1]
            label = self.label_to_idx[label]
            return image, label
        else:
            return image, img_name

### 数据增强与预处理

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### 载入数据

In [7]:
train_df, test_df, label_to_idx, idx_to_label = load_csv()
train_dataset = ButterflyDataset(train_df, TRAIN_IMAGES, label_to_idx, transform)
test_dataset = ButterflyDataset(test_df, TEST_IMAGES, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### 设计一个 CNN 模型的 class

In [8]:
class ButterflyCNN(nn.Module):
    def __init__(self, num_classes):
        super(ButterflyCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

### 实例化待训练模型，载入预训练的 Resnet18 卷积神经网络

In [9]:
model = ButterflyCNN(num_classes=len(label_to_idx)).to(DEVICE)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 211MB/s]


### 设计 Loss 函数和优化器

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

### 开始训练 (>_<)

In [11]:
def train_model():
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for images, labels in progress_bar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # 前向传播
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 反向传播与优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss / len(progress_bar))

### 开始预测 (^_^)

In [12]:
@torch.no_grad()
def predict():
    model.eval()
    predictions = []
    for images, img_names in tqdm(test_loader, desc="Predicting"):
        images = images.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels = [idx_to_label[idx.item()] for idx in predicted]
        predictions.extend(zip(img_names, predicted_labels))
    return predictions

# 保存预测结果，生成 submission.csv
def save_predictions(predictions):
    submission_df = pd.DataFrame(predictions, columns=['filename', 'label'])
    submission_df.to_csv(SUBMISSION_FILE, index=False)

In [13]:
if __name__ == "__main__":
    train_model()
    predictions = predict()
    save_predictions(predictions)
    print(f"Submission file saved as {SUBMISSION_FILE}")

Predicting: 100%|██████████| 102/102 [00:24<00:00,  4.08it/s]

Submission file saved as /kaggle/working/submission.csv
